In [5]:
import os
import gc
from tqdm import tqdm
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits import mplot3d
import math
import h5py


# %matplotlib widget
basePath = "./sims.TNG/TNG100-1/output"

initial_snapshot = 2
end_snapshot = 99

halos_fields = ["GroupFirstSub", "GroupNsubs", "GroupMass", "GroupPos"]
subhalos_fields = ["SubhaloGrNr", "SubhaloMass", "SubhaloPos"]
tree_fields = ["SubfindID", "SnapNum"]

h = 0.6774  # km/Mpc

m_dm = 4.5e-5  # E10M_Sum
m_dm_h = m_dm * h

Critial_Density = 1.27e-8  # E10M_Sum/kpc^3
Critial_Density_h = Critial_Density / h**2  # E10M_Sum/(kpc/h)^3


def Detect_Mkdir(Path):
    if not (os.path.exists(Path)):
        os.makedirs(Path)
    return Path


In [6]:
def Calculate_Halo_Dict(halo_index, host_index, mms_index):
    host_tree = il.sublink.loadTree(
        basePath, end_snapshot, host_index, fields=tree_fields, onlyMPB=1
    )
    mms_tree = il.sublink.loadTree(
        basePath, end_snapshot, mms_index, fields=tree_fields, onlyMPB=1
    )

    halo_indices_list = list()
    host_indices_list = list()
    mms_indices_list = list()

    host_mass_list = list()
    mms_mass_list = list()

    host_mass_type_list = list()
    mms_mass_type_list = list()

    satellite_radial_velocity_list = list()
    host_mms_distance = list()
    virial_radius_list = list()

    host_position_list = list()
    mms_position_list = list()
    host_velocity_list = list()
    mms_velocity_list = list()

    for current_snapshot in range(initial_snapshot, end_snapshot + 1):
        header = il.groupcat.loadHeader(basePath, current_snapshot)
        scale_factor = header["Time"]

        host_subhalo_index = host_tree["SubfindID"][
            np.where(host_tree["SnapNum"] == current_snapshot)[0]
        ]
        mms_subhalo_index = mms_tree["SubfindID"][
            np.where(mms_tree["SnapNum"] == current_snapshot)[0]
        ]

        if len(mms_subhalo_index) == 0 or len(host_subhalo_index) == 0:
            halo_indices_list.append(np.array(-1, dtype=np.int32))
            host_indices_list.append(np.array(-1, dtype=np.int32))
            mms_indices_list.append(np.array(-1, dtype=np.int32))

            host_mass_list.append(np.nan)
            mms_mass_list.append(np.nan)

            host_mass_type_list.append(
                np.array([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]))
            mms_mass_type_list.append(
                np.array([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]))

            satellite_radial_velocity_list.append(np.nan)
            host_mms_distance.append(np.nan)

            host_position_list.append(np.array([np.nan, np.nan, np.nan]))
            mms_position_list.append(np.array([np.nan, np.nan, np.nan]))

            host_velocity_list.append(np.array([np.nan, np.nan, np.nan]))
            mms_velocity_list.append(np.array([np.nan, np.nan, np.nan]))

            continue

        host_subhalo_dict = il.groupcat.loadSingle(
            basePath, current_snapshot, -1, host_subhalo_index
        )
        mms_subhalo_dict = il.groupcat.loadSingle(
            basePath, current_snapshot, -1, mms_subhalo_index
        )

        halo_dict = il.groupcat.loadSingle(
            basePath, current_snapshot, host_subhalo_dict['SubhaloGrNr'], -1)
        current_host_comoving_position = host_subhalo_dict["SubhaloPos"]
        current_mms_comoving_position = mms_subhalo_dict["SubhaloPos"]

        current_host_physical_position = current_host_comoving_position * scale_factor
        current_mms_physical_position = current_mms_comoving_position * scale_factor

        current_host_physical_position = current_host_physical_position % 35000
        current_mms_physical_position = current_mms_physical_position % 35000

        current_difference_physical_coordinates = (
            current_mms_physical_position - current_host_physical_position
        )
        distance = np.linalg.norm(current_difference_physical_coordinates)

        host_velocity = host_subhalo_dict["SubhaloVel"]
        mms_velocity = mms_subhalo_dict["SubhaloVel"]

        Vel_diff = mms_velocity - host_velocity
        Vel_Radius = (
            np.dot(Vel_diff, current_difference_physical_coordinates) / distance
        )
        halo_indices_list.append(np.array(host_subhalo_dict['SubhaloGrNr']))
        host_indices_list.append(host_subhalo_index[0])
        mms_indices_list.append(mms_subhalo_index[0])

        host_mass_list.append(host_subhalo_dict['SubhaloMass'])
        mms_mass_list.append(mms_subhalo_dict['SubhaloMass'])

        host_mass_type_list.append(host_subhalo_dict['SubhaloMassType'])
        mms_mass_type_list.append(mms_subhalo_dict['SubhaloMassType'])

        host_velocity_list.append(host_velocity)
        mms_velocity_list.append(mms_velocity)

        satellite_radial_velocity_list.append(Vel_Radius)
        host_mms_distance.append(distance)
        virial_radius_list.append(halo_dict['Group_R_Crit200'])

        host_position_list.append(current_host_physical_position)
        mms_position_list.append(current_mms_physical_position)
        gc.collect()

    del host_tree, mms_tree, header, scale_factor
    del host_subhalo_dict, host_subhalo_index
    del mms_subhalo_dict, mms_subhalo_index
    del current_host_comoving_position, current_mms_comoving_position
    del current_host_physical_position, current_mms_physical_position
    del current_difference_physical_coordinates, distance
    del host_velocity, mms_velocity, Vel_diff, Vel_Radius

    halo_indices_array = np.array(halo_indices_list)
    host_indices_array = np.array(host_indices_list)
    mms_indices_array = np.array(mms_indices_list)

    host_mass_array = np.array(host_mass_list)
    mms_mass_array = np.array(mms_mass_list)

    host_mass_type_array = np.array(host_mass_type_list)
    mms_mass_type_array = np.array(mms_mass_type_list)

    host_velocity_array = np.array(host_velocity_list)
    mms_velocity_array = np.array(mms_velocity_list)

    satellite_radial_radius_velocity = np.array(satellite_radial_velocity_list)
    host_mms_distance = np.array(host_mms_distance)
    virial_radius_array = np.array(virial_radius_list)

    host_position_array = np.array(host_position_list)
    mms_position_array = np.array(mms_position_list)

    del satellite_radial_velocity_list, host_position_list, mms_position_list

    return_data = dict()

    return_data["Halo_Index_0"] = halo_index
    return_data["Host_Index_0"] = host_index
    return_data["MMSG_Index_0"] = mms_index

    return_data["Halo_Index"] = halo_indices_array
    return_data["Host_Index"] = host_indices_array
    return_data["MMSG_Index"] = mms_indices_array

    return_data["Host_Mass"] = host_mass_array
    return_data["MMSG_Mass"] = mms_mass_array

    return_data["Host_MassType"] = host_mass_type_array
    return_data["MMSG_MassType"] = mms_mass_type_array

    return_data["Radial_Velocity"] = satellite_radial_radius_velocity
    return_data["Distance"] = host_mms_distance
    return_data["Virial_Radius"] = virial_radius_array

    return_data["Host_Position"] = host_position_array
    return_data["MMS_Position"] = mms_position_array

    return_data["Host_Velocity"] = host_velocity_array
    return_data["MMSG_Velocity"] = mms_velocity_array

    return_data["Snapshots"] = np.arange(initial_snapshot, end_snapshot + 1)

    del satellite_radial_radius_velocity, host_mms_distance, host_position_array, mms_position_array
    del host_indices_array, mms_indices_array, host_mass_array, mms_mass_array, host_mass_type_array, mms_mass_type_array, host_velocity_array, mms_velocity_array
    gc.collect()

    return return_data


In [3]:
halos_dict = il.groupcat.loadHalos(basePath, end_snapshot, fields=halos_fields)
subhalos_dict = il.groupcat.loadSubhalos(
    basePath, end_snapshot, fields=subhalos_fields)


In [4]:
galaxy_mass_inf = 30
galaxy_mass_sup = 500
condition = (halos_dict["GroupMass"] >= galaxy_mass_inf) & (
    halos_dict["GroupMass"] <= galaxy_mass_sup)
mw_like_group_index = np.where(condition)[0]
mw_like_host_index = np.array([], dtype=int)
mw_like_mmsg_index = np.array([], dtype=int)
for i in tqdm(mw_like_group_index):
    Firstsubid = halos_dict['GroupFirstSub'][i]
    if Firstsubid == -1:
        continue
    if halos_dict['GroupNsubs'][i] == 1:
        continue
    Mass_array = np.array([])
    for j in range(halos_dict['GroupNsubs'][i]):
        Subhalo_id = Firstsubid+j
        Mass_array = np.append(
            Mass_array, subhalos_dict['SubhaloMass'][Subhalo_id])
    mw_like_host_index = np.append(
        mw_like_host_index, Firstsubid+(np.argsort(Mass_array)[-1]))
    mw_like_mmsg_index = np.append(
        mw_like_mmsg_index, Firstsubid+(np.argsort(Mass_array)[-2]))


100%|██████████| 4396/4396 [00:02<00:00, 1486.99it/s]


In [7]:
hdf5_File_Path = './APMS_TNG100_hdf5_z12'

start_sub_index = len(os.listdir(Detect_Mkdir(hdf5_File_Path)))-1
step_length = 200
end = start_sub_index+step_length

start_sub_index+=2*step_length
end+=2*step_length

if end > mw_like_group_index.shape[0]:
    end = mw_like_group_index.shape[0]

if start_sub_index < 0:
    start_sub_index = 0

for Subindex in tqdm(range(start_sub_index, end)):
    halo_index = mw_like_group_index[Subindex]
    host_index = mw_like_host_index[Subindex]
    mmsg_index = mw_like_mmsg_index[Subindex]

    halo_dict = Calculate_Halo_Dict(halo_index, host_index, mmsg_index)

    with h5py.File((os.path.join(hdf5_File_Path, 'Halo-'+str(halo_dict['Halo_Index_0']))+'.hdf5'), 'w') as f:
        for key, value in halo_dict.items():
            f.create_dataset(key, data=value)


 14%|█▎        | 27/200 [11:58<1:18:19, 27.16s/it]

: 

In [155]:
hdf5_File_Path = './APMS_TNG100_hdf5_z12'
hdf5_files = [f for f in os.listdir(hdf5_File_Path) if f.endswith('.hdf5')]
for file_name in hdf5_files:
    with h5py.File(os.path.join(hdf5_File_Path, file_name), 'r') as f:
        for key in f.keys():
            globals()[key] = f[key][()]
    break
